# SCENARI D'USO

In [1]:
import carla, time, threading, random
from agents.navigation.basic_agent import BasicAgent
from Adas import Forward_collision_warning_mqtt
from SyncSimulation import SyncSimulation

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# FUNZIONI DI SUPPORTO

## Gestione simulazione

In [37]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 0.8

## ADAS

In [66]:
 # per Audi TT
target_vehicle_id = 2
target_vehicle_ttc = 0

In [39]:
def attach_adas(sim, world, vehicle, min_ttc = 0.5):
    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        min_ttc=min_ttc,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : sim.stop_simulation()
    )
    return adas

## Gestione world

In [40]:
def load_map(world, map_name):
    if not world.get_map().name == f"Carla/Maps/{map_name}":
        try:
            world = client.load_world(map_name)
        except RuntimeError:
            print("")

In [41]:
def destroy_all_vehicles(world):
    all_actors = world.get_actors()
    
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    for vehicle in vehicles:
        vehicle.destroy()

In [42]:
def destroy_all_pedestrians(world):
    all_actors = world.get_actors()
    
    pedestrians = all_actors.filter('walker.pedestrian.*')
    
    print(f"Numero di pedoni trovati: {len(pedestrians)}")
    
    for pedestrian in pedestrians:
        pedestrian.destroy()

In [43]:
def disable_traffic_lights():
    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.set_state(carla.TrafficLightState.Green)
        traffic_light.set_green_time(99999)
        traffic_light.freeze(True)

In [44]:
def enable_traffic_lights():

    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.freeze(False)
        traffic_light.reset_group()
        print(f"Semaforo {traffic_light.id} ripristinato al comportamento predefinito.")

## Creazione entità

In [45]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    world.wait_for_tick()
    return vehicle

In [46]:
def actor_vehicle(world, vehicle, desitination, speed = 60):
    actor_agent = BasicAgent(vehicle)
    actor_agent.set_destination(desitination)
    actor_agent.set_target_speed(speed)
    actor_agent.ignore_vehicles()
    world.wait_for_tick()
    return actor_agent

In [47]:
def spawn_walker(start_location, yaw = 0):
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
    walker_bp.set_attribute('is_invincible', 'false')

    trans = carla.Transform()
    trans.location = start_location
    trans.rotation = carla.Rotation(yaw = yaw)

    return world.spawn_actor(walker_bp, trans)

## Gestione movimenti

In [48]:
def move_forward(vehicle, throttle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

In [49]:
def move_agent(vehicle, actor_agent):
    vehicle.apply_control(actor_agent.run_step())

In [50]:
def move_forward_agent(vehicle, actor_agent,x,y):
    forward_location = vehicle.get_location() + carla.Location(x=x, y=y, z=0)  # Adjust 'x', 'y' based on vehicle's orientation
    actor_agent.set_destination(forward_location)
    vehicle.apply_control(actor_agent.run_step())

In [51]:
def stop_vehicle(vehicle):
    print("stop vehicle")
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    control.hand_brake = True
    vehicle.apply_control(control)

In [52]:
def pedestrian_control(x=1,y =0,z = 0):
    walker_control = carla.WalkerControl()
    walker_control.speed = 1.3  # Velocità in m/s (positivo in avanti)
    walker_control.direction = carla.Vector3D(x, y, z)
    return walker_control

In [53]:
def stop_pedestrian(walker):
    walker_control = carla.WalkerControl()
    walker_control.speed = 0  # Velocità in m/s (positivo in avanti)
    walker.apply_control(walker_control)

In [54]:
def _move_pedestrian(walker, controller_w, sim, extra_time):
    while sim.run:
        walker.apply_control(controller_w)
        time.sleep(0.5)
    time.sleep(extra_time)
    stop_pedestrian(walker)

def move_pedestrian(walker, controller_w, sim, extra_time):
    threading.Thread(target=_move_pedestrian, args=(walker, controller_w, sim, extra_time,)).start()

## Camera

In [55]:
def print_spectator_position():
    location = world.get_spectator().get_transform().location
    rotation = world.get_spectator().get_transform().rotation
    print(location)
    print(rotation)

In [56]:
def set_spectator_location(location, rotation):
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(location, rotation))

In [57]:
def run_camera(camera, vehicle, sim):
    return threading.Thread(target=camera, args=(vehicle,sim))

In [58]:
def view_from_above(vehicle, sim):    
    while sim.run:
        distance_above = 40.0 
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        
        
        spectator_location = carla.Location(
                x=vehicle_location.x, 
                y=vehicle_location.y,
                z=vehicle_location.z + distance_above
                )

        spectator_rotation = carla.Rotation(
            pitch=-90.0,
            yaw=0,
            roll=0.0
        )
        
        spectator.set_transform(carla.Transform(spectator_location, spectator_rotation))
    

In [59]:
def view_from_behind(vehicle, sim):    
    while sim.run:
        distance_behind = 14.0 
        height = 5.0
        distance_to_the_side = 0
        
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        vehicle_rotation = vehicle.get_transform().rotation
        
        spectator_location = carla.Location(
                x=vehicle_location.x - distance_behind * vehicle_rotation.get_forward_vector().x + distance_to_the_side * vehicle_rotation.get_right_vector().x,
                y=vehicle_location.y - distance_behind * vehicle_rotation.get_forward_vector().y + distance_to_the_side * vehicle_rotation.get_right_vector().y,
                z=vehicle_location.z + height
            )
    
        spectator.set_transform(carla.Transform(spectator_location, vehicle_rotation))
    
    

# SCENARI

## Scenario 0: Viaggio senza ostacoli

Il veicolo segue un percorso prestabilito senza presenza di ostacoli. Lo scopo è quello di  verificare che in condiziondi normali il veicolo viaggi tranquillamente.

### SETUP

In [26]:
load_map(world, "Town10HD_Opt")

In [27]:
disable_traffic_lights()

# percorso da seguire
wps = [
    carla.Location(x=-41.679836, y=45.000507, z=0),
    carla.Location(x=30.000090, y=28.267391, z=0),
    carla.Location(x=-41.525066, y=100.000069, z=0)

]

start = carla.Location(x=-41.477230, y=116.999939, z=1)
vehicle_speed = 40
destination = wps[0]

### RUN

In [ ]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, target_vehicle_id, -90)

actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

wp_index = 0

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()
    run_camera(view_from_behind, vehicle, simulation).start()
    
    while simulation.run:
        if actor_agent.done():
            wp_index = (wp_index +1) % len(wps)
            actor_agent.set_destination(wps[wp_index])

        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


## Scenario 1: Veicolo fermo o parcheggiato sulla carreggiata


Presenza di un veicolo parcheggiato direttamente davanti lungo la corsia di marcia. In questo scenario vi è la presenza di un veicolo posto ad una distanza: vicina, intermedia e lontada dal veicolo

### SETUP

In [29]:
load_map(world, "Town10HD_Opt")

In [30]:
disable_traffic_lights()

close_location = carla.Location(x=-41.5, y=100, z=1)
middle_location = carla.Location(x=-41.5, y=70, z=1)
far_location = carla.Location(x=-41.5, y=45, z=1)

start = carla.Location(x=-41.5, y=117, z=1)
vehicle_speed = 60
destination = far_location

### RUN

In [ ]:
destroy_all_vehicles(world)
simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, target_vehicle_id, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, middle_location, target_vehicle_id, -90)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()
    run_camera(view_from_above, vehicle, simulation).start()

    while simulation.run:
        move_agent(vehicle, actor_agent)

    stop_vehicle(vehicle)
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()

finally:
    print("done")
    adas.destroy()

## Senario 2: Veicolo davanti in movimento con una velocità uguale
Presenza di un veicolo davanti al veicolo di interesse, con un'accellerazione pari a quella del veicolo di interesse

### SETUP

In [53]:
load_map(world, "Town10HD_Opt")

In [66]:
disable_traffic_lights()

start_vehicle = carla.Location(x=-41.5, y=115, z=1)
start_obstacle = carla.Location(x=-41.5, y=100, z=1)
destination = carla.Location(x=-41.647892, y=-42.639099, z=0.000000)

vehicle_speed = 40
obstacle_speed = 40

### RUN

In [ ]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start_vehicle, target_vehicle_id, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, start_obstacle, target_vehicle_id, -90)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, obstacle_speed)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()  
    run_camera(view_from_above, vehicle, simulation).start()
    while simulation.run:
        if obstacle_actor_agent.done():
            stop_vehicle(obstacle_vehicle)
        move_agent(vehicle, actor_agent)
        move_agent(obstacle_vehicle, obstacle_actor_agent)
    
    stop_vehicle(vehicle)
    simulation.stop_simulation()
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(obstacle_vehicle)
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


## Senario 3: Veicolo davanti in movimento con una velocità minore

### SETUP

In [32]:
load_map(world, "Town10HD_Opt")

In [33]:
disable_traffic_lights()

start = carla.Location(x=-41.5, y=115, z=1)
obstacle_start = carla.Location(x=-41.5, y=95, z=1)
destination = carla.Location(x=-41.5, y=40, z=1)

vehicle_speed = 50
obstacle_speed = 30

### RUN

In [ ]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)


vehicle = spawn_vehicle(world, start, target_vehicle_id, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, obstacle_start, target_vehicle_id, -90)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, obstacle_speed)

try:
    adas = attach_adas(simulation, world, vehicle, 0.4)
    simulation.run_sync_simulation()  
    run_camera(view_from_above, vehicle, simulation).start()
    while True:
        if obstacle_actor_agent.done():
            break
        if simulation.run:
            move_agent(vehicle, actor_agent)
        else:
            stop_vehicle(vehicle)
            
        move_agent(obstacle_vehicle, obstacle_actor_agent)
    
    stop_vehicle(obstacle_vehicle)
    simulation.stop_simulation()
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    adas.destroy()
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


## Scenario 4 : Pedone che attraversa le strisce pedonali


Presenza di un pedone che attraversa le strisce pedonali mentre il veicolo di riferimento è a pochi metri da esse

### SETUP

In [36]:
load_map(world, "Town10HD_Opt")

In [37]:
disable_traffic_lights()

### Pedone in mezzo alla strada fermo

In [38]:
start = carla.Location(x=-41.5, y=80, z=1)
vehicle_speed = 120
destination = carla.Location(x=-41.5, y=10, z=1)

pedestrian_position = carla.Location(x=-41.5, y=37, z=1)

In [ ]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)

simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, target_vehicle_id, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

walker = spawn_walker(pedestrian_position)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()
    set_spectator_location(carla.Location(x=-37.506443, y=33.672970, z=1.671275),
        carla.Rotation(pitch=-6.648561, yaw=118.152756, roll=-0.000091))

    while simulation.run:
        move_agent(vehicle, actor_agent)
            
    stop_vehicle(vehicle)
    
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()

### Pedone in movimento

In [40]:
start = carla.Location(x=-41.5, y=80, z=1)
vehicle_speed = 40
destination = carla.Location(x=-41.5, y=10, z=1)

pedestrian_position = carla.Location(x=-35, y=36.6, z=1)

In [ ]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)


simulation = SyncSimulation(world)


vehicle = spawn_vehicle(world, start, target_vehicle_id, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

walker = spawn_walker(pedestrian_position, 180)
controller_w = pedestrian_control(-1,0,0)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()

    set_spectator_location(carla.Location(x=-35.991619, y=30.547363, z=2.301113),
        carla.Rotation(pitch=-6.984530, yaw=120.027725, roll=-0.000092))
    
    move_pedestrian(walker, controller_w, simulation, 2)

    while simulation.run:
        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()



## Scenario 5: Salita

### SETUP

In [67]:
load_map(world, "Town03")

In [72]:
disable_traffic_lights()

start = carla.Location(x=178.759857, y=-5.810725, z=1)
vehicle_speed = 40

close_location = carla.Location(x=152.025681, y=-21.958912, z=1)
middle_location = carla.Location(x=152.256927, y=-33.253975, z=3)
far_location = carla.Location(x=152.688828, y=-58.084942, z=7)

destination = far_location

### RUN

In [ ]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, target_vehicle_id, 180)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, far_location, target_vehicle_id, -90)
stop_vehicle(obstacle_vehicle)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()
    run_camera(view_from_behind, vehicle, simulation).start()

    while simulation.run:
        move_agent(vehicle, actor_agent)

    stop_vehicle(vehicle)
    time.sleep(1)
    set_spectator_location(carla.Location(x=139.538391, y=-9.965685, z=15.529643),
                           carla.Rotation(pitch=-35.153622, yaw=-40.927944, roll=0))
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()
    

## Scenario 6: Discesa

### SETUP

In [63]:
load_map(world, "Town03")

In [64]:
disable_traffic_lights()

start = carla.Location(x=149.401398, y=-100.336784, z=9)
vehicle_speed = 40
destination = carla.Location(x=111.904289, y=-6.809386, z=0.119281)

close_location = carla.Location(x=149.237808, y=-56.931606, z=7)
middle_location = carla.Location(x=149.052383, y=-40.271507, z=6)
far_location = carla.Location(x=149.052383, y=-20.271507, z=2)

### RUN

In [ ]:
destroy_all_vehicles(world)
simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, target_vehicle_id, 90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, far_location, 0, 90) 

stop_vehicle(obstacle_vehicle)

try:
    adas = attach_adas(simulation, world, vehicle, target_vehicle_ttc)
    simulation.run_sync_simulation()
    run_camera(view_from_behind, vehicle, simulation).start()
    
    while simulation.run:
        if actor_agent.done():
            simulation.stop_simulation()

        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)
    time.sleep(1)
    set_spectator_location(carla.Location(x=142.891983, y=-48.781475, z=48.636322),
        carla.Rotation(pitch=-82.175034, yaw=-2.961209, roll=0.695270))

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()